# Lab: Linear Regression + Model Evaluation
Drew Malter, Matt Winkler    
Slalom    
July 2018    

## Overview
The goal of this lab is to provide a hands-on introduction to the main tools/methods for creating and evaluating supervised learning regression models. This lab covers well established regression practices along with statistical reasoning as it works through a problem from start to finish.

The examples here utilize data from a large car insurance provider seeking to predict the amount a customer is willing to spend on an insurance policy with given information about each customer.  The benefits of these predictions are extremely valuable to both the sales agents and the marketing team. 

Sales Agents: Provides them the crucial advatnage of knowing what a customer is probably willing to spend as they help their customers find a plan

Marketing: Allows them to focus their marketing campaigns on an audience that will spend more money, or attempt to move a low spending customer base into a higher spending customer base.

In [ ]:
#Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import seaborn as sns
import os

%matplotlib inline

from ipywidgets import interact, widgets

from bokeh.io import push_notebook, show, output_notebook, curdoc
from bokeh.plotting import figure
from bokeh.models.annotations import Title
from bokeh.models.widgets import MultiSelect
from bokeh.models import Button
from bokeh.layouts import widgetbox, column, row

In [ ]:
#load in the data set
#Import 3 data sets; orders, prices, and customers
os.chdir('C:/Users/drew.malter/Documents/AIML Workshop/SalesDatasets')
df = pd.read_csv('CarInsurance_Claims.csv')

In [ ]:
#add descriptions to each data field

Inspection

In [ ]:
#Look at the first 5 rows of the data set
df.head()

In [ ]:
#Return dimension of table
df.shape

In [ ]:
# List data types
#note: write line of code to convert those that are necessary
df.dtypes

In [ ]:
# Check to see if there is any missing data.  There are many practices for dealing with missing data if true
#Decide what to do about the missing data
#I like to input the median value given one or two contraints
#but since this is a large data set we can afford to just remove those rows
df.isnull().any()

In [ ]:
#drop Na's
df = df.dropna()
df.shape

In [ ]:
#Drop non numeric fields that we will not need
df = df.drop(columns = ["A","B","C","D","E","F","G", "customer_ID", "shopping_pt", "record_type", "day", "location"])

In [ ]:
#Look at the summary statistics of each numeric field
df.describe()

Some features appear to show skew based on the output above, since the means are higher than the medians.  We will visually confirm this in a few ways and address how to deal with skew

In [ ]:
#Plot a histogram of car age, note that there are large outliers inflating the mean higher than the median
plt.hist(df['car_age'], density=True, bins=30)
plt.ylabel('Probability')
plt.xlabel('Car Age');

The Bokeh library provides an interactive interface for examining each feature in the data.

In [ ]:
# plot setup:
p = figure(title="Test", 
           background_fill_color="#E8DDCB", 
           width=500, 
           height=300)

# get values and make histogram:
arr = df['car_age'].values
hist, edges = np.histogram(arr, 
                           density=True, 
                           bins=50)

# Setup x axis based on the data:
#arr_min = min(arr)
#arr_max = max(arr)
#n_unique = len(np.unique(arr))
#x = np.linspace(arr_min, arr_max, n_unique)

# assign histogram to the figure:
r = p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],
        fill_color="#036564", line_color="#033649")

# labelling:
#p.legend.location = "center_right"
#p.legend.background_fill_color = "darkgrey"
#p.xaxis.axis_label = 'x'
#p.yaxis.axis_label = 'Pr(x)'

# show static plot:
#show(p)

In [ ]:
# update() function to make the plot interactive:
def update(feature, num_bins):
    # update array and histogram values:
    arr = df[feature].values
    hist, edges = np.histogram(arr, 
                           density=True, 
                           bins=num_bins)
    
    # update plot parameters:
    p.title.text = 'Probability distribution of {}'.format(feature)
    p.xaxis.axis_label = '{}'.format(feature)
    p.yaxis.axis_label = 'Pr({})'.format(feature)
    
    # reset the renderer data:
    r.data_source.data['top'] = hist
    r.data_source.data['left'] = edges[:-1]
    r.data_source.data['right'] = edges[1:]
    
    push_notebook()

In [ ]:
#once converted to numeric variables there will be less options
features = df.columns.values
nbins = (10, 50)
interact(update, feature = features, num_bins = nbins)
show(p, notebook_handle = True)

In [ ]:
# example boxplot for the group size feature. TODO: make dynamic with bokeh
sns.boxplot(df['car_age'])

In [ ]:
#one possible outlier removal method, lets talk more about this
# define outlier adjustment function:
def adjust_outliers(array_in):
    """ Finds and replaces outliers in input array with 1.5 * IQR"""
    array_out = np.array(array_in)
    
    q1 = array_in.quantile(q=0.25)
    q3 = array_in.quantile(q=0.75)
    iqr = q3 - q1
    upper = q3 + 1.5*iqr
    lower = q1 - 1.5*iqr
    
    below_ind = array_out < lower
    above_ind = array_out > upper
    
    array_out[below_ind] = lower
    array_out[above_ind] = upper
    
    changed = np.sum(array_in != array_out)
    print('Found and adjusted {} values in {} column'.format(str(changed), array_in.name))
    return array_out

In [ ]:
#need to fix this
# Adjust for outliers across the dataset, except for cost column since it's the output:
names = list(df.columns.values)
names.remove('cost')
df_adj = df[names].apply(adjust_outliers)
df_adj['cost'] = df['cost']

In [ ]:
#once above line is fixed
# confirm this worked:
sns.boxplot(df_adj['car_age'])

In [ ]:
# Look at correlation matrix of numeric variables:

# Compute the correlation matrix
corr = df.corr()

# Generate a mask for the upper triangle
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(220, 10, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})

In [ ]:
# Build model:
from sklearn.model_selection import train_test_split
from sklearn import linear_model
lm = linear_model.LinearRegression()

In [ ]:
#Set target to Y, set predictors to X
y = df_adj.cost
X = df_adj.drop('cost', axis=1)

#Split data set into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=101)

#Fit the model :
lm.fit(X_train, y_train)

#Set predictions
lm_train_pred = lm.predict(X_train)
lm_test_pred = lm.predict(X_test)

In [ ]:
# Do a quick PCA for plotting and compare predictions to dummy regressor:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

from sklearn.dummy import DummyRegressor

regdummy = DummyRegressor(strategy="mean")
regdummy.fit(X_train, y_train)
dummy_y = regdummy.predict(X_train)

scaler = StandardScaler()
X_s = scaler.fit_transform(X_train)

pca = PCA(n_components=1)
pca.fit(X_s)
X_pca = pca.fit_transform(X_s)

m, b = np.polyfit(X_pca[:,0], lm_train_pred, 1)
plt.plot(X_pca, m*X_pca + b, "-", color="palevioletred")
plt.axhline(y=regdummy.constant_, linestyle="-", color="limegreen")
plt.scatter(X_pca[:,0], lm_train_pred)
plt.ylabel("Predicted Value")
plt.xlabel("Observed X variables index")
plt.title("Comparison of Regression with Mean model")

In [ ]:
#Return the summary of the model
X2 = sm.add_constant(X)
est = sm.OLS(y, X2)
est2 = est.fit()
print(est2.summary())

In [ ]:
y_mn = np.mean(y_train)
sst = sum((y_train - y_mn)**2)
ssr = sum((lm_train_pred - y_train)**2)
r2 = 1 - (ssr / sst)
r2

In [ ]:
y = df_adj.target
X_names = [nm for nm in df_adj.columns.values if nm != 'cost']
X = df_adj.drop("cost", axis=1)

def make_predictions(X, y, X_names):
    """Subset X to selected features, then use to predict y"""
    X = X[X_names]
    #Split data set into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=101)
    
    #Fit the model:
    lm.fit(X_train, y_train)
    
    return X_train, y_train, lm.predict(X_train)
    
X_train, y_train, lm_train_pred = make_predictions(X, y, X_names)

def make_plot_arrays(X_train, lm_train_pred):
    """Get first PCA component from X_train for plotting only"""
    # fitting PCA:
    X_s = scaler.fit_transform(X_train)
    pca = PCA(n_components=1)
    pca.fit(X_s)
    X_pca = pca.fit_transform(X_s)
    # make plot lines:
    m, b = np.polyfit(X_pca[:,0], lm_train_pred, 1)
    x_arr = X_pca[:, 0]
    
    # normalize so the scale doesn't change with different sets of features
    x_arr_norm = ( (x_arr - x_arr.min()) / (x_arr.max() - x_arr.min()) )
    
    y_arr = m*X_pca[:, 0] + b
    
    return x_arr_norm, y_arr

x_arr, y_arr = make_plot_arrays(X_train, lm_train_pred)

def calc_r2(y_train, preds):
    y_mn = np.mean(y_train)
    sst = sum((y_train - y_mn)**2)
    ssr = sum((y_train - preds)**2)
    
    r2 = 1 - (ssr / sst)
    return np.round(r2, 4)

baseline_r2 = calc_r2(y_train, dummy_y)
pred_r2 = calc_r2(y_train, lm_train_pred)

p = figure(title="Regression model vs. baseline", 
           plot_height=300, 
           plot_width=600, 
           y_range=(0,60))

# scatterplot of actual data:
r1 = p.circle(x_arr, y_train, size=10, color="green", alpha=0.5)
# scatterplot of prediction data:
r2 = p.circle(x_arr, lm_train_pred, size=10, color="pink", alpha=0.5)
# regression model:
r3 = p.line(x_arr, y_arr, color="#2222aa", line_width=3)
# baseline dummy model:
p.line(x_arr, dummy_y, color="purple", line_width=2)

p.xaxis.axis_label = 'X feature array index'
p.yaxis.axis_label = 'Predicted price'

#show(p)

In [ ]:
X_opts = [nm for nm in df_adj.columns.values if nm != 'cost']

m = widgets.SelectMultiple(
    options=X_opts,
    value=X_opts,
    rows=10,
    description='Features',
    disabled=False
)

def update_regression(features):
    X_selected = list(features)
    
    # update predictions based on selected features:
    X_train, y_train, lm_train_pred = make_predictions(X, y, X_selected)
    x_arr, y_arr = make_plot_arrays(X_train, lm_train_pred)
    
    # update plots:
    r1.data_source.data['x'] = x_arr
    r1.data_source.data['y'] = y_train
    
    r2.data_source.data['x'] = x_arr
    r2.data_source.data['y'] = lm_train_pred
    
    r3.data_source.data['x'] = x_arr
    r3.data_source.data['y'] = y_arr
    
    pred_r2 = calc_r2(y_train, lm_train_pred)
    baseline_r2 = calc_r2(y_train, dummy_y)
    
    print('Predicted R^2: {}'.format(pred_r2))
    print('Baseline R^2: {}'.format(baseline_r2))
    push_notebook()
    
interact(update_regression, features = m)
show(p, notebook_handle=True)